# Content:¶
Goal of this notebook is to support the business to harmonize the manufacturer names we have in the different data sources, eg.
IHS, SE and TC have different naming convention for the the same manufacturer and potentially even within a single data source. Inspiration is the fuzzy string search also used in the similarity mapping.
The final table should look like this:

| Manufacturer_ORG | Alternative_1|Alternative_2|Alternative_3|Alternative_4|Alternative_5| 
| --- | --- | --- | --- | --- | --- |
| AVX | AVX | AVX Company | AXV comp| NAVXCY | Somthing Else | Something weird|

In [26]:
# Import packages (necessary for nbconvert runs)
%run packages.py
if globals()['pd']: 
    print("start script successfully loaded all packages")

start script successfully loaded all packages


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
import re
import itertools
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
# import pickle
#from udf import read_raw_rables, clean_tables, make_read_data_url, create_att_table
import multiprocessing as mp
from tqdm import tqdm
import os
import sys
import requests
from itertools import product
from requests.auth import HTTPBasicAuth
import json
from collections import Counter
# Add source script to read data from Hana
sys.path.append('/home/Z000SYPA/hana-r-python-connecticity/python/')
from src.udf import make_read_data_url

## Set global variables

In [28]:
# GLOBAL VARIABLES
prod_run = True if ('prod' in os.popen('hostname').read().rstrip()[-4:]) else False # Checks if running on prod instance
hana_sys = 'q' # Can either be '0', 'q' or '1'
use_cache = True # If True, then intermediate results are stored/read on/from S3
writeback_to_hana = True

s3_path = "/S3/FILES_MT_S3/"
datapath = os.path.join(os.path.dirname(os.getcwd()), "data")
print(datapath)
if not os.path.exists(datapath):
    os.makedirs(datapath)

/home/ubuntu/matspec_ai/data


In [29]:
pattern = r'(\band\b|\bcorp\b|\bkg\b|gmbh|handelsgesellschftmbH|technologies|\belectronic\b|\belectronics\b|ltd|\binc\b|\bco\b|llc|limited|corporation|company|holdings|\bkg\b|\bag\b|[^a-z])'

## Define odata HANA settings

In [30]:
# Get username and password for HANA
HANA_username = ("%s" % os.popen('/opt/parameter-get user').read()).rstrip('\n')
if (hana_sys == '0'):
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh0').read()).rstrip('\n')
else:
    HANA_password = ("%s" % os.popen('/opt/parameter-get password_eh1').read()).rstrip('\n')

In [31]:
# Read parameters

# Create URL to CV
if (hana_sys == '1'):
    main_url = 'https://hanadatalake.siemens.com/siemens/MATSPEC/AWS/'
else:
    main_url = 'https://hanadatalake-eh' + hana_sys + '.siemens.com/siemens/MATSPEC/AWS/'

xsodata_hana_to_py = 'ODATA_CONNECTIONS.xsodata'
#source_object = '/T_IHSDOC'

## Read data from HANA

In [32]:
manufacturer_to_read = '/V_MASTERDATA_PARTS_MANUFACTURERS'
cmd_manufacturer = '/V_CMD_DATA'

In [33]:
url_manufacturer = make_read_data_url(main_url, xsodata_hana_to_py,manufacturer_to_read)
url_cmd = make_read_data_url(main_url, xsodata_hana_to_py,cmd_manufacturer)

print(url_manufacturer)  
print(url_cmd)

https://hanadatalake-ehq.siemens.com/siemens/MATSPEC/AWS/ODATA_CONNECTIONS.xsodata/V_MASTERDATA_PARTS_MANUFACTURERS?$format=json
https://hanadatalake-ehq.siemens.com/siemens/MATSPEC/AWS/ODATA_CONNECTIONS.xsodata/V_CMD_DATA?$format=json


In [34]:
result = requests.get(url=url_manufacturer, auth=HTTPBasicAuth(HANA_username, HANA_password), verify=False)
df_manu = pd.read_json(json.dumps(result.json()['d']['results']), orient='list')
            #df_list.append(data.drop(columns = ['GenID','__metadata'], inplace = True))
df_manu.drop(columns = ['GenID','__metadata'], inplace = True)
df_manu.head()

/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


PART_ID      MANUFACTURER_NAME DATA_SOURCE_ID
0          ADVANCED MICRODEVICES             TC
1           AGW Electronics Ltd.             TC
2                   ALFATEC GMBH             TC
3             ALTERA CORPORATION             TC
4              AMI Semiconductor             TC

In [35]:
result = requests.get(url=url_cmd, auth=HTTPBasicAuth(HANA_username, HANA_password), verify=False)
#df_cmd = pd.read_json(json.dumps(result.json()['d']['results']), orient='list')

/opt/anaconda/envs/shared/lib/python3.7/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [36]:
df_cmd = pd.read_json(json.dumps(result.json()['d']['results']), orient='list')

In [37]:
df_cmd.drop(columns = ['GenID','__metadata'], inplace = True)
df_cmd.head()

COMPANYNAME_GU
0                       " OBAL PRESS, spol. s r.o. "
1           " SWMS Systemtechnik Ingenieurges. mbH "
2               " TERMOELEKTRO - NS " d.o.o.Gradiska
3  ""LS 78 MANTES EN YVELINES"" LOISIRS SPORTIFS ...
4                                        "A M C" SRL

In [38]:
def name_cleaner(pattern, list_name):
    """
        cleans every pattern occurence in every list element and returns a cleaned list with every first letter in capital
    """
    # replace pattern
    choices_clean = [re.sub(pattern, " ",  name.lower()) for name in list_name]   
    # replace additional whitespaces
    choices_clean = [re.sub("\s+", " ", name.strip()) for name in choices_clean if len(name) > 2]
    # remove words that are only up to 2 character
    tmp = [[word for word in name.split() if len(word) > 2] for name in choices_clean]
    choices_clean = [" ".join(name) for name in tmp]
    # replace names that are in total < 2
    choices_clean = [name.strip() for name in choices_clean if len(name) > 2]
    choices_clean = [name.title() for name in choices_clean]
    return list(set(choices_clean))
   

In [39]:
df_cmd["MANUFACTURER_NAME_CLEAN"] = (
    df_cmd
    .COMPANYNAME_GU
    .str.lower()
    .str.replace(pattern, " ", regex = True)
    .str.strip()
    .apply(lambda x: " ".join([word for word in x.split() if len(word) > 2]))
    .str.title()
)
df_cmd.head()

COMPANYNAME_GU  \
0                       " OBAL PRESS, spol. s r.o. "   
1           " SWMS Systemtechnik Ingenieurges. mbH "   
2               " TERMOELEKTRO - NS " d.o.o.Gradiska   
3  ""LS 78 MANTES EN YVELINES"" LOISIRS SPORTIFS ...   
4                                        "A M C" SRL   

                            MANUFACTURER_NAME_CLEAN  
0                                   Obal Press Spol  
1               Swms Systemtechnik Ingenieurges Mbh  
2                             Termoelektro Gradiska  
3  Mantes Yvelines Loisirs Sportifs Mantes Yvelines  
4                                               Srl

In [40]:
df_manu["MANUFACTURER_NAME_CLEAN"] = (
    df_manu
    .MANUFACTURER_NAME   
    .str.lower()
    .str.replace(pattern, " ", regex = True)
    .str.strip()
    .apply(lambda x: " ".join([word for word in x.split() if len(word) > 2]))
    .str.title()
)

In [41]:
df_manu.drop(columns="PART_ID", inplace = True)
df_manu.drop_duplicates(inplace = True)
df_manu.shape

(5174, 3)

In [48]:
df_manu.head()

MANUFACTURER_NAME DATA_SOURCE_ID MANUFACTURER_NAME_CLEAN
0  ADVANCED MICRODEVICES             TC   Advanced Microdevices
1   AGW Electronics Ltd.             TC                     Agw
2           ALFATEC GMBH             TC                 Alfatec
3     ALTERA CORPORATION             TC                  Altera
4      AMI Semiconductor             TC       Ami Semiconductor

In [43]:
size = len(df_manu)//15
list_of_dfs = [df_manu.loc[i:i+size-1,:] for i in range(0, len(df_manu))]

## Define specific classes to be used in sklearn pipeline

In [44]:
def find_CMD_match(comp,df,column_to_check, scorer = None):
    if not scorer:
        scorer = fuzz.token_set_ratio
    
        
    highest = process.extractOne(comp,list(df[column_to_check]),scorer = scorer)
    idx = df[df[column_to_check] == highest[0]].index
    #print(idx)
    if len(idx)>1:
        index = idx[0]
    else:
        index = idx
    if type(df_cmd.iloc[index,0]) == str:
        result = None, None
    else:
        result = df_cmd.iloc[index,0].tolist()[0], highest[1]
    return result

In [45]:
def run_fuzzy_string_match(df):
    df["CMD_BEST_MATCH"] = (
        df
        .MANUFACTURER_NAME_CLEAN
        .apply(lambda x: find_CMD_match(x,df_cmd, "MANUFACTURER_NAME_CLEAN" ) )   
    )
    
    
    df[['CMD_BEST_MATCH', 'MATCH_SCORE']] = pd.DataFrame(df['CMD_BEST_MATCH'].tolist(), index=df.index)
    return df

In [47]:
results = Parallel(n_jobs=-1, verbose = 1)(delayed(run_fuzzy_string_match,)(df) for df in list_of_dfs)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed: 81.4min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed: 297.3min


KeyboardInterrupt: 

In [ ]:
df_manufacturer_mapping = pd.concat(results)
df_manufacturer_mapping.drop(columns = "MANUFACTURER_NAME_CLEAN", inplace = True)
df_manufacturer_mapping.head()

In [ ]:
df_manufacturer_mapping.shape

### Store result as excel

In [ ]:
df_manufacturer_mapping.to_csv('/S3/FILES_MT_S3/df_manufacturer_mapping.csv', sep = "|", index = False)